In [8]:
# pip install selenium

In [11]:
# Bibek Shiwakoti



In [ ]:
""" import os
import csv
import requests
from bs4 import BeautifulSoup

headers = {
    "User-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

# Skills and Place of Work
skill = input('Enter your Skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the #pages to scrape: '))


# Creating the Main Directory
main_dir = os.getcwd() + '\\'
if not os.path.exists(main_dir):
    os.mkdir(main_dir)
    print('Base Directory Created Successfully.')


# Name of the CSV File
file_name = skill.title() + '_' + place.title() + '_Jobs.csv'
# Path of the CSV File
file_path = main_dir + file_name

# Writing to the CSV File
with open(file_path, mode='w') as file:
    writer = csv.writer(file, delimiter=',', lineterminator='\n')
    # Adding the Column Names to the CSV File
    writer.writerow(
        ['JOB_NAME', 'COMPANY', 'LOCATION', 'POSTED', 'APPLY_LINK'])

    # Requesting and getting the webpage using requests
    print(f'\nScraping in progress...\n')
    for page in range(no_of_pages):
        url = 'https://www.indeed.co.in/jobs?q=' + skill + \
            '&l=' + place + '&start=' + str(page * 10)
        response = requests.get(url, headers=headers)
        html = response.text

        # Scrapping the Web
        soup = BeautifulSoup(html, 'lxml')
        base_url = 'https://in.indeed.com/viewjob?jk='
        d = soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

        jobs = soup.find_all('a', class_='tapItem')

        for job in jobs:
            job_id = job['id'].split('_')[-1]
            job_title = job.find('span', title=True).text.strip()
            company = job.find('span', class_='companyName').text.strip()
            location = job.find('div', class_='companyLocation').text.strip()
            posted = job.find('span', class_='date').text.strip()
            job_link = base_url + job_id
            #print([job_title, company, location, posted, job_link])

            # Writing to CSV File
            writer.writerow(
                [job_title, company, location.title(), posted, job_link])

print(f'Jobs data written to <{file_name}> successfully.')  """

In [8]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import pandas as pd 
import time

def generate_url(jobTitle,location):
    """Generate URL given job title and location"""
    url = f"https://ca.indeed.com/{jobTitle}-jobs-in-{location}"
    return url

def get_record(card):
    """Extract info from one job card"""
    
    atag = card.h2.a
    jobTitle = atag.get('title')
    jobURL = "https://ca.indeed.com" + atag.get('href')
    jobCompany = card.find('span',class_='company').text.strip()
    
    # for job location, sometimes it's <span> sometimes it's <div> 
    if card.find('span',class_='location'):
        jobLocation = card.find('span',class_='location').text.strip()
    else:
        jobLocation = card.find('div',class_='location').text.strip()
        
    jobSummary = card.find('div','summary').text.strip()
    jobPostDate = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    
    # sometimes there's salary
    if card.find('span','salaryText'):
        jobSalary = card.find('span','salaryText').text.strip() 
    else:
        jobSalary = ''
        
    job = (jobTitle,jobCompany,jobLocation,jobPostDate,today,jobSalary,jobSummary,jobURL)
    
    return job

In [12]:
# should maybe rotate
user_agent = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.3 Safari/605.1.15'}

records = []

job_title = "python"  # empty string for all jobs
loc = "Canada"
url = generate_url(jobTitle = job_title, location = loc)
print(url)
# there will be pop-ups but that doesn't matter
while True:
    time.sleep(1)
    response = requests.get(url, user_agent)
    soup = BeautifulSoup(response.text,'html.parser')
    cards = soup.find_all('div',class_='job_seen_beacon')
    
    for card in cards: 
        record = get_record(card)
        records.append(record)    # append tuple to list
        
    try:
        url = "https://ca.indeed.com" + soup.find('a',{'aria-label':'Next'}).get('href')
    except AttributeError:
        break


https://ca.indeed.com/python-jobs-in-Canada


In [13]:
# make dataframe 
records_df = pd.DataFrame(records, columns =['jobTitle','jobCompany','jobLocation','jobPostDate','today','jobSalary','jobSummary','jobURL']) 

records_df

,jobTitle,jobCompany,jobLocation,jobPostDate,today,jobSalary,jobSummary,jobURL
